In [3]:
from netCDF4 import Dataset
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import subprocess
import os
%matplotlib inline

## Exploring NVDI data

In [58]:
data_folder = '../data/nvdi/nc/'
files = os.listdir(data_folder)

In [107]:
print 'Number of files =', len(files)
print 'Example file name:', files[0]
print 'Shape of all latitudes', lats.shape
print 'Shape of all longitudes', lons.shape
print 'Sahep of NVDI variable', var.shape

Number of files = 449
Example file name: 2002.01.01.mask.nc
Shape of all latitudes (614,)
Shape of all longitudes (927,)
Sahep of NVDI variable (614, 927)


In [81]:
# Loading the first NDVI netCDF file to import lat lon variables
nc = Dataset(data_folder+files[0], 'r')
lats = nc.variables['lat'][:]
lons = nc.variables['lon'][:]
nc.close()

# Load all the files in the data folder
data = np.zeros([len(lats), len(lons), len(files)])
for i, f in enumerate(files):
    nc = Dataset(data_folder+f, 'r')
    var = nc.variables['Band1'][:]
    data[:,:,i] = var
    nc.close()
    
print data.shape

In [83]:
# Constructing a composite mask where True == somewhere in the time series there's a NaN value
# -3000 is the null values used for NDVI files downloaded from USGS
is_nan = (data == -3000.)
mask = np.any(is_nan, axis=2)
print mask.shape

(614, 927)

## Useful plotting functions

In [126]:
def plot_time_series(i, j, data):
    time_series = data[i, j, :]
    fig = plt.figure(figsize=(20,6))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(np.arange(len(time_series)), time_series)
    note = 'lat = {0}, lon = {1}'.format(lats[i], lons[j])
    ax.text(0.9, 2, note, fontsize=20)
    plt.show()

In [ ]:
plot_time_series(200, 400, data)

## Feature engineering from the weather data

In [21]:
# Aggregate from daily data to 8-day data to match NDVI

folder = '../data/weather/'
yrs = np.arange(2002, 2012)
t = 8 # getting 8 day stats to match NDVI data

# Setting the variables to use
variables = ['Prec', 'Tmax', 'Tmax', 'Tmin', 'Tmin', 'Wind']
stats     = ['sum' , 'max',  'mean',  'min', 'mean', 'max']
func = 'timsel{s},{t}'
infile = folder + '{yr}.{v}.nc'
outfile = folder + '{yr}.{s}.of.{v}.nc'

for v, s in zip(variables, stats):
    file_list = []
    for yr in yrs:
        if yr == 2002:
            subprocess.call(['cdo', 'timsel{s},{t},176'.format(s=s, t=t), infile.format(yr=yr, v=v), outfile.format(yr=yr, v=v, s=s)])
        else:
            subprocess.call(['cdo', func.format(s=s, t=t), infile.format(yr=yr, v=v), outfile.format(yr=yr, v=v, s=s)])
        file_list.append(outfile.format(yr=yr, v=v, s=s))
        
    subprocess.call(['cdo', 'mergetime', ' '.join(file_list), folder+'all.{s}.of.{v}.nc'.format(v=v, s=s)])
    
    subprocess.call(['cdo', 'splitsel,437', folder+'all.{s}.of.{v}.nc'.format(v=v, s=s), folder+'all.{s}.of.{v}.nc'.format(v=v, s=s)])
    subprocess.call(['mv', folder+'all.{s}.of.{v}.nc000000.nc'.format(v=v, s=s), folder+'all.{s}.of.{v}.nc'.format(v=v, s=s)])
    subprocess.call(['rm', folder+'all.{s}.of.{v}.nc000001.nc'.format(v=v, s=s)])